In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_contrast(image, lower=0.2, upper=2.0)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=0.2, upper = 0.2)
    image = tf.image.random_hue(image, max_delta = 0.08)
    image = tf.image.random_saturation(image, lower=0.2, upper=0.2)
    return image, label
    

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same',bias_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax', bias_regularizer=tf.keras.regularizers.l2(0.01))
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        3

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(3).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=20,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 10:33:17.729442: {'loss': 1.1294663419882456, 'accuracy': 0.59915, 'val_loss': 0.9429279736652496, 'val_accuracy': 0.6728}
Epoch 1 ends at 10:34:21.072964: {'loss': 0.6876917716503144, 'accuracy': 0.7624417, 'val_loss': 0.9091906585511128, 'val_accuracy': 0.6985}
Epoch 2 ends at 10:35:28.156329: {'loss': 0.528450136423111, 'accuracy': 0.81673336, 'val_loss': 1.229917755172511, 'val_accuracy': 0.6533}
Epoch 3 ends at 10:36:40.972333: {'loss': 0.4236197190284729, 'accuracy': 0.85350835, 'val_loss': 0.9199793541887004, 'val_accuracy': 0.7288}
Epoch 4 ends at 10:38:00.436124: {'loss': 0.3412747973243396, 'accuracy': 0.8815917, 'val_loss': 0.8614044829159025, 'val_accuracy': 0.7641}
Epoch 5 ends at 10:39:22.750047: {'loss': 0.2877398352642854, 'accuracy': 0.90065, 'val_loss': 0.9238156194140197, 'val_accuracy': 0.764}
Epoch 6 ends at 10:40:41.918354: {'loss': 0.2464600383847952, 'accuracy': 0.91595834, 'val_loss': 0.8712231962923791, 'val_accuracy': 0.7737}
Epoch 7 ends at 1

In [12]:
model.evaluate(test_data)

157/157 [==============================]0.9834 - accuracy: 0.765 - 0s 70ms/step - loss: 1.0832 - accuracy: 0.781 - 0s 56ms/step - loss: 1.0418 - accuracy: 0.80 - 0s 49ms/step - loss: 1.0014 - accuracy: 0.81 - 0s 52ms/step - loss: 0.9657 - accuracy: 0.81 - 0s 49ms/step - loss: 0.9659 - accuracy: 0.82 - 0s 47ms/step - loss: 1.0549 - accuracy: 0.81 - 0s 45ms/step - loss: 1.1771 - accuracy: 0.80 - 0s 44ms/step - loss: 1.1382 - accuracy: 0.81 - 0s 43ms/step - loss: 1.1856 - accuracy: 0.80 - 0s 42ms/step - loss: 1.1558 - accuracy: 0.81 - 0s 41ms/step - loss: 1.1750 - accuracy: 0.81 - 1s 41ms/step - loss: 1.2015 - accuracy: 0.80 - 1s 40ms/step - loss: 1.2132 - accuracy: 0.80 - 1s 40ms/step - loss: 1.1942 - accuracy: 0.80 - 1s 39ms/step - loss: 1.2311 - accuracy: 0.80 - 1s 39ms/step - loss: 1.2332 - accuracy: 0.79 - 1s 39ms/step - loss: 1.2816 - accuracy: 0.79 - 1s 38ms/step - loss: 1.2737 - accuracy: 0.79 - 1s 38ms/step - loss: 1.2781 - accuracy: 0.79 - 1s 38ms/step - loss: 1.2351 - accuracy:

[1.2612651859166895, 0.7961]

In [13]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))